<a href="https://colab.research.google.com/github/tarboledah/SyS/blob/main/Parcial2/Parcial_punto4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 975.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.
     - 2.7 MB 9.4 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylin

In [2]:
%%writefile Punto_4.py
#Código Dashboard
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

st.title("PUNTO 4")
st.header("Calculo de Distorsion total de armonicos")
# Parámetros del sistema
sampling_rate = 10000  # Frecuencia de muestreo
T = 1.0 / sampling_rate  # Intervalo de tiempo
t = np.linspace(0, 1.0, sampling_rate)

# Generar señal del rectificador de onda completa
def rectificador_onda_completa(t, f, R, C=None):
    # Onda senoidal fundamental
    signal = np.abs(np.sin(2 * np.pi * f * t))

    if C is not None:
        # Respuesta para carga RC
        tau = R * C
        signal = signal * np.exp(-t / tau)

    return signal

# Calcular la FFT y los armónicos
def calcular_fft(signal, sampling_rate):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    return xf, np.abs(yf[:N // 2])

# Calcular THD
def calcular_thd(yf):
    V1 = yf[1]  # El primer armónico
    harmonics = np.sqrt(np.sum(yf[2:]**2))
    thd = harmonics / V1
    return thd

# Parámetros de la simulación
st.subheader("Parámetros de la simulación")
f = 60  # Frecuencia de la señal de entrada (60 Hz)
R = float(st.number_input("Ingrese el valor de R deseado:", value=10.1))  # Valor de la resistencia (ohmios)
C = float(st.number_input("Ingrese el valor de C deseado:", value=1.1))  # Valor del condensador (faradios)

# Simulación para carga resistiva pura
signal_resistiva = rectificador_onda_completa(t, f, R)

# Simulación para carga RC en serie
signal_rc = rectificador_onda_completa(t, f, R, C)

# FFT de ambas señales
xf_res, yf_res = calcular_fft(signal_resistiva, sampling_rate)
xf_rc, yf_rc = calcular_fft(signal_rc, sampling_rate)

# Calcular THD
thd_resistiva = calcular_thd(yf_res)
thd_rc = calcular_thd(yf_rc)

st.write(f"THD para carga resistiva pura: {thd_resistiva:.4f}")
st.write(f"THD para carga RC: {thd_rc:.4f}")

# Graficar señales y su espectro
plt.figure(figsize=(12, 6))

# Señal en el dominio del tiempo
plt.subplot(2, 2, 1)
plt.plot(t[:500], signal_resistiva[:500])
plt.title("Señal con carga resistiva pura")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")

plt.subplot(2, 2, 2)
plt.plot(t[:500], signal_rc[:500])
plt.title("Señal con carga RC")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")

# FFT de las señales
plt.subplot(2, 2, 3)
plt.plot(xf_res, yf_res)
plt.title("FFT - Carga resistiva pura")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Amplitud")

plt.subplot(2, 2, 4)
plt.plot(xf_rc, yf_rc)
plt.title("FFT - Carga RC")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Amplitud")

plt.tight_layout()
st.pyplot()


Writing Punto_4.py


In [3]:
!streamlit run Punto_4.py &>/content/logs.txt & curl ipv4.icanhazip.com #Password del Dashboard

35.237.85.139


In [4]:
!npx localtunnel --port 8501 #Link del Dashboard

your url is: https://all-mammals-allow.loca.lt
^C
